In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def huber_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    epsilon=1.35,
    alpha=0.0001,
    max_iter=1000,
    use_cv=False,
    cv_params=None,
    random_state=None
):
    """
    Robust Huber regression for time series forecasting with:
    - Automatic hyperparameter tuning via grid search
    - Outlier-resistant modeling
    - Enhanced differencing checks
    """
    # Create directory structure
    os.makedirs('HuberRegression', exist_ok=True)
    
    # 1. Stationarity and Differencing (fixed adf_result check)
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:  # Fixed p-value access
            d = 1
            series = series.diff().dropna()

    # 2. Feature Engineering - Lag features with type consistency
    df = pd.DataFrame({'y': series.astype(np.float64)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(np.float64)
    df = df.dropna()

    # 3. Time-based split with validation
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 4. Model training with optional CV
    if use_cv:
        # Default parameter grid
        param_grid = cv_params or {
            'epsilon': [1.0, 1.35, 1.5, 2.0],
            'alpha': [0.0001, 0.001, 0.01, 0.1]
        }
        
        grid_search = GridSearchCV(
            HuberRegressor(max_iter=max_iter),
            param_grid,
            cv=5,
            scoring='neg_mean_squared_error'
        )
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
        best_epsilon = model.epsilon
        best_alpha = model.alpha
    else:
        model = HuberRegressor(
            epsilon=epsilon,
            alpha=alpha,
            max_iter=max_iter
        )
        model.fit(X_train, y_train)
        best_epsilon = epsilon
        best_alpha = alpha

    # 5. Forecasting with type consistency
    pred_test = model.predict(X_test).astype(np.float64)

    # 6. Inverse differencing with edge case handling
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 7. Enhanced DataFrame with confidence intervals
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast': pred_test,
        'residual': y_test.values - pred_test
    })

    # 8. Robust CSV saving
    forecast_csv_path = os.path.join('HuberRegression', 'huber_forecasts.csv')
    forecast_df.to_csv(
        forecast_csv_path,
        mode='a',
        header=not os.path.exists(forecast_csv_path),
        index=False
    )
     # 9. Comprehensive metrics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))
    metrics_data = {
        'district': district_name,
        'rmse': rmse,
        'r_squared': model.score(X_train, y_train),
        'coefficients': model.coef_.tolist(),
        'intercept': model.intercept_,
        'final_epsilon': best_epsilon,
        'final_alpha': best_alpha,
        'differencing': d,
        'n_iterations': model.n_iter_
    }

    if use_cv:
        metrics_data.update({
            'best_params': grid_search.best_params_,
            'cv_results': grid_search.cv_results_
        })

    metrics_df = pd.DataFrame([metrics_data])
    
    metrics_csv_path = os.path.join('HuberRegression', 'huber_metrics.csv')
    metrics_df.to_csv(
        metrics_csv_path,
        mode='a',
        header=not os.path.exists(metrics_csv_path),
        index=False
    )

    # 10. Enhanced plotting with residuals
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Forecast plot
    ax1.plot(original_series.index, original_series.values, label='Original Series')
    ax1.plot(forecast_df['date'], forecast_df['forecast'], 
            label='Huber Forecast', color='darkorange')
    ax1.set_title(f'Huber Regression Forecast for {district_name}\nRMSE: {rmse:.2f}')
    ax1.legend()

    # Residual plot
    ax2.bar(forecast_df['date'], forecast_df['residual'], 
           color='gray', alpha=0.7, width=5)
    ax2.axhline(0, color='red', linestyle='--')
    ax2.set_title('Forecast Residuals')
    
    plt.tight_layout()
    plt.savefig(os.path.join('HuberRegression', f'huber_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'model_params': model.get_params(),
        'feature_names': X_train.columns.tolist()
    }

In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I40"].asfreq('MS')
        
        results = huber_regression_lags_only(
            ts,
            district,
            use_cv=True,
            cv_params={
                'epsilon': [1.0, 1.35, 1.7],
                'alpha': [0.0001, 0.001, 0.01]
            },
            max_iter=2000
        )
        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

=== Metrics ===
     district       rmse  r_squared  \
0  AHMEDNAGAR  18.124588  -0.108051   

                                        coefficients  intercept  \
0  [0.555249904436399, 0.17262177856385144, 0.216...   1.726207   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.0        0.001             0            24   

                        best_params  \
0  {'alpha': 0.001, 'epsilon': 1.0}   

                                          cv_results  
0  {'mean_fit_time': [0.029792928695678712, 0.010...  

=== Forecast Data ===
     district       date  actual   forecast   residual
0  AHMEDNAGAR 2020-07-01    40.0  36.727437   3.272563
1  AHMEDNAGAR 2020-08-01    55.0  38.448925  16.551075
2  AHMEDNAGAR 2020-09-01    36.0  47.598668 -11.598668
3  AHMEDNAGAR 2020-10-01    82.0  39.854371  42.145629
4  AHMEDNAGAR 2020-11-01    49.0  65.357916 -16.357916
=== Metrics ===
  district      rmse  r_squared  \
0    AKOLA  9.059014   0.046188   

                  

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
     district       rmse  r_squared  \
0  AURANGABAD  14.981617   0.045813   

                                        coefficients  intercept  \
0  [-0.2459448135510169, 0.08255031794327211, -0....   0.202396   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.7         0.01             1            26   

                       best_params  \
0  {'alpha': 0.01, 'epsilon': 1.7}   

                                          cv_results  
0  {'mean_fit_time': [0.023499679565429688, 0.005...  

=== Forecast Data ===
     district       date  actual   forecast   residual
0  AURANGABAD 2020-07-01      53  60.940422  -7.940422
1  AURANGABAD 2020-08-01      74  64.163726   9.836274
2  AURANGABAD 2020-09-01      76  58.261031  17.738969
3  AURANGABAD 2020-10-01      56  59.869018  -3.869018
4  AURANGABAD 2020-11-01      57  64.811171  -7.811171


c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
  district     rmse  r_squared  \
0     BEED  7.61874   0.287831   

                                        coefficients  intercept  \
0  [0.5495369149808582, -0.010527871446260547, -0...  16.667139   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.7         0.01             0            40   

                       best_params  \
0  {'alpha': 0.01, 'epsilon': 1.7}   

                                          cv_results  
0  {'mean_fit_time': [0.019092226028442384, 0.004...  

=== Forecast Data ===
  district       date  actual   forecast   residual
0     BEED 2020-07-01    30.0  23.690009   6.309991
1     BEED 2020-08-01    30.0  27.791574   2.208426
2     BEED 2020-09-01    33.0  28.981970   4.018030
3     BEED 2020-10-01    16.0  28.978249 -12.978249
4     BEED 2020-11-01    18.0  19.604538  -1.604538
=== Metrics ===
   district      rmse  r_squared  \
0  BHANDARA  4.224697   0.087732   

                                        coeffi

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
  district      rmse  r_squared  \
0   RAIGAD  9.116054   0.460354   

                                        coefficients  intercept  \
0  [0.7368008699746859, 0.09143508292796972, -0.2...   9.484513   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.0       0.0001             0            28   

                         best_params  \
0  {'alpha': 0.0001, 'epsilon': 1.0}   

                                          cv_results  
0  {'mean_fit_time': [0.013079595565795899, 0.004...  

=== Forecast Data ===
  district       date  actual   forecast   residual
0   RAIGAD 2020-07-01    21.0  28.676891  -7.676891
1   RAIGAD 2020-08-01    29.0  22.462043   6.537957
2   RAIGAD 2020-09-01    35.0  27.187334   7.812666
3   RAIGAD 2020-10-01    32.0  33.580629  -1.580629
4   RAIGAD 2020-11-01    11.0  30.264159 -19.264159


c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
    district      rmse  r_squared  \
0  RATNAGIRI  5.719087   0.122883   

                                        coefficients  intercept  \
0  [0.24541179262340032, 0.31602627426369784, -0....   7.896062   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.0       0.0001             0            26   

                         best_params  \
0  {'alpha': 0.0001, 'epsilon': 1.0}   

                                          cv_results  
0  {'mean_fit_time': [0.012117767333984375, 0.003...  

=== Forecast Data ===
    district       date  actual   forecast   residual
0  RATNAGIRI 2020-07-01    23.0  15.811674   7.188326
1  RATNAGIRI 2020-08-01    25.0  13.431560  11.568440
2  RATNAGIRI 2020-09-01    22.0  18.415108   3.584892
3  RATNAGIRI 2020-10-01    11.0  16.456376  -5.456376
4  RATNAGIRI 2020-11-01    19.0  12.396645   6.603355
=== Metrics ===
  district     rmse  r_squared  \
0   SANGLI  8.25409   0.024106   

                            

{'district': 'MUMBAI SUBURBAN',
 'forecast_df':           district       date  actual   forecast   residual
 0  MUMBAI SUBURBAN 2021-02-01      28  32.777410  -4.777410
 1  MUMBAI SUBURBAN 2021-03-01      28  38.594869 -10.594869,
 'metrics_df':           district      rmse  r_squared  \
 0  MUMBAI SUBURBAN  8.218117   0.886475   
 
                                         coefficients  intercept  \
 0  [-1.2779124100110162, -0.21314708059436546, 0....   -5.98405   
 
    final_epsilon  final_alpha  differencing  n_iterations  \
 0            1.0       0.0001             1            41   
 
                          best_params  \
 0  {'alpha': 0.0001, 'epsilon': 1.0}   
 
                                           cv_results  
 0  {'mean_fit_time': [0.0232027530670166, 0.02002...  ,
 'model_params': {'alpha': 0.0001,
  'epsilon': 1.0,
  'fit_intercept': True,
  'max_iter': 2000,
  'tol': 1e-05,
  'warm_start': False},
 'feature_names': ['lag_1', 'lag_2', 'lag_3']}